In [10]:
import pandas as pd
import requests
import os

Code for downloading Udacity's Dog Prediction Data

In [12]:
#url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
#r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

In [16]:
# make a directory if one does not already exist
#folder_name = 'dog_predictions'
#if not os.path.exists(folder_name):
    #os.makedirs(folder_name)

In [23]:
#with open(os.path.join(folder_name, 
                           #url.split('/')[-1]), mode='wb') as file:
             #file.write(r.content)

In [25]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
twitter_archive;

In [26]:
predicts = pd.read_csv('dog_predictions/image-predictions.tsv', sep='\t')
predicts;